In [2]:
from  pyspark.sql import  SparkSession, functions as f

# Creating Spark session
spark =SparkSession.builder\
                   .appName("HealthCare Claims Ingestion")\
                   .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/23 08:52:13 INFO org.apache.spark.SparkEnv: Registering MapOutputTracker
25/08/23 08:52:13 INFO org.apache.spark.SparkEnv: Registering BlockManagerMaster
25/08/23 08:52:13 INFO org.apache.spark.SparkEnv: Registering BlockManagerMasterHeartbeat
25/08/23 08:52:13 INFO org.apache.spark.SparkEnv: Registering OutputCommitCoordinator


In [14]:
#configuring variables
BUCKET_NAME = "healthcare-bucket-14082025"
CLAIMS_BUCKET_PATH = f"gs://{BUCKET_NAME}/landing/claims/*.csv"
BQ_TABLE = "infra-memento-469009-a5.bronze_dataset.claims"
TEMP_GCS_BUCKET = f"{BUCKET_NAME}/temp"



#reading from claims source
claims_df = spark.read.csv(CLAIMS_BUCKET_PATH, header = True)

claims_df = claims_df.withColumn(
    "filename",
    f.regexp_extract(f.input_file_name(), r"([^/]+$)", 1)
)

claims_df = claims_df.dropDuplicates()\
                     .withColumn(
                                 "datasource",
                                 f.when(f.col("filename").like("%hospital2%"), "hosb")
                                 .when(f.col("filename").like("%hospital1%"), "hosa")
                                 .otherwise("None")
                                )

claims_df = claims_df.drop("filename")

claims_df.show(5, truncate=False)


+-----------+-------------+------------+-----------+----------+-------+-----------+----------+----------------+-----------+----------+-----------+----------+----------+-----------+-----+----------+------------+----------+
|ClaimID    |TransactionID|PatientID   |EncounterID|ProviderID|DeptID |ServiceDate|ClaimDate |PayorID         |ClaimAmount|PaidAmount|ClaimStatus|PayorType |Deductible|Coinsurance|Copay|InsertDate|ModifiedDate|datasource|
+-----------+-------------+------------+-----------+----------+-------+-----------+----------+----------------+-----------+----------+-----------+----------+----------+-----------+-----+----------+------------+----------+
|CLAIM000054|TRANS006567  |HOSP1-001387|ENC001081  |PROV0194  |DEPT008|2024-06-25 |2024-07-27|Medicaid        |3430.7     |2409.03   |Rejected   |Private   |114.75    |88.9       |7.49 |2021-07-28|2024-08-30  |hosb      |
|CLAIM000339|TRANS004459  |HOSP1-002776|ENC004331  |PROV0180  |DEPT017|2024-02-17 |2024-06-07|BlueCross       |2

In [15]:
#writing to BigQuery
claims_df.write\
         .format("bigquery")\
         .option("table", BQ_TABLE)\
         .option("temporaryGcsBucket", TEMP_GCS_BUCKET)\
         .mode("overwrite")\
         .save()